In [1]:
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("movies_metadata.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df1 = df[["id", "title"]]

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45466 non-null  object
 1   title   45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [6]:
df1 = df1.dropna(axis=0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45460 non-null  object
 1   title   45460 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [7]:
df1.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [8]:
df1["id"] = df1["id"].astype(int)

In [9]:
df1.set_index('id', inplace = True)

In [10]:
print(df1)

                              title
id                                 
862                       Toy Story
8844                        Jumanji
15602              Grumpier Old Men
31357             Waiting to Exhale
11862   Father of the Bride Part II
...                             ...
439050                       Subdue
111109          Century of Birthing
67758                      Betrayal
227506             Satan Triumphant
461257                     Queerama

[45460 rows x 1 columns]


In [12]:
# Generate mapping between titles and index
indices = pd.Series(df1.index, index=df1['title'])

In [13]:
print(indices)

title
Toy Story                         862
Jumanji                          8844
Grumpier Old Men                15602
Waiting to Exhale               31357
Father of the Bride Part II     11862
                                ...  
Subdue                         439050
Century of Birthing            111109
Betrayal                        67758
Satan Triumphant               227506
Queerama                       461257
Name: id, Length: 45460, dtype: int64


In [14]:
Overview = df["overview"].head(1000)

In [15]:
Overview = Overview.dropna()

Recomendation function

In [19]:
def get_recommendations(title, cosine_sim, indices):
    # Get index of movie that matches title
    idx = indices[title]
    # Sort the movies based on the similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return df1['title'].iloc[movie_indices]

IMPORT 

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

Vectorization , Cosine similarity , recommendation

In [21]:
# Initialize the TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words='english')

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(Overview)

# Generate the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Generate recommendations 
print(get_recommendations('Batman Forever', cosine_sim, indices))

id
36593                 The Naked Gun 33⅓: The Final Insult
9089                                Home for the Holidays
854                                              The Mask
9090     To Wong Foo, Thanks for Everything! Julie Newmar
22317                                      Lightning Jack
89333                                             Catwalk
11827                                         Heavy Metal
11907                                     Nobody Loves Me
274                              The Silence of the Lambs
10412                                      Romper Stomper
Name: title, dtype: object
